**PROBLEM DESCRIPTION**

This notebook demostrates a **sequence classification of IMDB movie reviews** dataset by creating a simple LSTM based classifier. 

Each movie review is a variable sequence of words, and the tone of each movie review must be classified. The large movie reviews dataset (sometimes referred to as the IMDB dataset) contains *25,000 film reviews (good or bad) for training and 25,000 reviews for testing*. The problem is deciding whether a given movie review is positive or negative. The data were collected by researchers at Stanford  and  used in a 2011 paper that used 50-50  data  for training and testing. An accuracy of 88.89% is achieved. 


**Import modules**

Let's start off with the basic step of importing all the relevant modules and functions required for this particular classifier.

In [111]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model


Firstly, let's learn about our dataset. For this we need to import the data and convert it into a Pandas' dataframe.

In [92]:
df = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

df.head()

**Load and Clean Dataset**

In the original dataset, the reviews are still dirty. There are still html tags, numbers, uppercase, and punctuations. This will not be good for training, so in load_dataset() function, beside loading the dataset using pandas, I also pre-process the reviews by removing html tags, non alphabet (punctuations and numbers), stop words, and lower case all of the reviews.

In [93]:
english_stops = set(stopwords.words('english')) #declaring stop words

def load_dataset():
    df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

The total review count in above data is - 

In [94]:
df.sentiment.value_counts()

The data can be visualized by using features of matplotlib library. By doing so, we can the following results,

In [95]:
Sentiment_count=df.groupby('sentiment').count()
plt.bar(Sentiment_count.index.values, Sentiment_count['review'])
plt.xlabel('Review Sentiments')
plt.ylabel('Number of Review')
plt.show()

Next, take the amount of data you want to use for this particular model. The original dataset contains 50,000 movie reviews. But, it can be restricted as per the requirement. 

Then split the dataset into train (70%) and test (30%) sets.

In [106]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.3)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

In [107]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))


**Tokenize and Pad/Truncate Reviews**

A Neural Network only accepts numeric data, so we need to encode the reviews.

Each reviews has a different length, so we need to add padding (by adding 0) or truncating the words to the same length (in this case, it is the mean of all reviews length) using tensorflow.keras.preprocessing.sequence.pad_sequences.


In [108]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

**Define the LSTM model**

The first layer is the Embedded layer that uses 32 length vectors to represent each word. The next layer is the LSTM layer with 100 memory units (smart neurons). Subsequently, you can add more than one LSTM layer. Finally, because this is a classification problem we use a Dense output layer with a single neuron and a sigmoid activation function to make 0 or 1 predictions for the two classes (good and bad) in the problem.

Because it is a *binary classification problem*, log loss is used as the loss function (binary_crossentropy in Keras). The efficient ADAM optimization algorithm is used. The number of epochs and batch size can be increased as per the requirement. 

In [112]:
# LSTM Model
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())


In [115]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

# Final evaluation of the model

model.fit(x_train, y_train, batch_size = 256, epochs = 5, callbacks=[checkpoint])

Once the model is created, then we can test the performance on unseen reviews. 

In [117]:
y_pred = model.predict(x_test, batch_size = 128)

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))